In [5]:
import os
import string
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [4]:
### Lower characters
def lowerChar(ip):
    s=""
    for ch in ip:
        ch=ch.lower()
        s+=ch
    return s

In [6]:
#--------------------------------------TOKENIZATION-----------------------------#

def tokenize_ipStr(ip):   
    temp=word_tokenize(ip)
    return temp  

In [7]:
#### Remove Stop words

def removeStopwords(temp):
    stoWords=set(stopwords.words('english'))
    s=""
    ans=[]
    for h in temp:
        if not h in stoWords:
            ans.append(h)
    return ans 

In [8]:
### Remove Punctuations
def removePuntuations(ip):
    test_str = ip.translate(str.maketrans('', '', string.punctuation))
    return test_str
    

In [9]:
#### Remove White Spaces
def removeWS(ip):
    s1=" ".join(ip.split())
    return s1

In [10]:
## Preprocessing
def preprocess(ip):
    # call lower function
    s=lowerChar(ip)
    s=removePuntuations(s)
    s=removeWS(s)
    # call tokenize file function
    temp = tokenize_ipStr(s)
    #Call remove Stop words funtions
    #print("\n\n\n")
    s=removeStopwords(temp)
    
    return s

In [11]:
####### OR Function 
def orFunc(l1,l2):
    lenl1 = len(l1)
    lenl2 = len(l2)
    i=0
    j=0
    ansList = []
    cmprs = 0
    while(i<lenl1 and j<lenl2):
        if(l1[i] == l2[j]):
            ansList.append(l1[i])
            i=i+1
            j=j+1
        elif(l1[i] < l2[j]):
            ansList.append(l1[i])
            i=i+1
        else:
            ansList.append(l2[j])
            j=j+1

        cmprs=cmprs + 1

    while(i<lenl1):
        ansList.append(l1[i])
        i=i+1
    while(j<lenl2):
        ansList.append(l2[j])
        j=j+1
        
    return ansList,cmprs ##### MERGING

In [12]:
#### AND Function

def andFunc(l1,l2):
    lenl1 = len(l1)
    lenl2 = len(l2)
    i=0
    j=0
    ansList = []
    cmprs = 0
    while(i<lenl1 and j<lenl2):
        if(l1[i] == l2[j]):
            ansList.append(l1[i])
            i=i+1
            j=j+1
        elif(l1[i] < l2[j]):
            i=i+1
        else:
            j=j+1

        cmprs=cmprs + 1

    return ansList,cmprs ##### MERGING

In [13]:
### NOT Func

def notFunc(list):
    ansList = []
    cmprs=0
    for i in range(1,1401):
        if i not in list:
            ansList.append(i)

    return ansList,cmprs

In [14]:
### AND NOT Function

def andNot(l1,l2):
    tempList = notFunc(l2)
    ansList,cmprs = andFunc(l1,tempList)
    return ansList,cmprs

In [15]:
### OR NOT Function

def orNot(l1,l2):
    tempList = notFunc(l2)
    ansList,cmprs = orFunc(l1,tempList)
    return ansList,cmprs

In [16]:
######## Unigram inverted index ##########

def unigramInvIndex():
    with open('D:/SEMESTER-2/IR-Ass1/file.pkl', 'rb') as file:
        
        # Call load method to deserialze
        myvar = pickle.load(file)
  
    return myvar

In [17]:
doctIdDict={}
for i in range(1,1401):
    s=""
    if(i>=1 and i<=9):
        s="000"
    elif(i>=10 and i<=99):
        s="00"
    if(i>=100 and i<=999):
        s="0"
    text="cranfield"+s+"{x}".format(x=i)
    doctIdDict[i]=text


In [20]:
####### User Input ###########

def userInput():
    print("Enter Query: ")
    ip = input()
    ans = preprocess(ip) ##### Preprocess
    print("Enter {temp} Operations: ".format(temp=len(ans)-1))
    operList=[]
    for i in range(0,len(ans)-1):
        op=input()
        operList.append(op)
    print("Tokens :", ans)
    print("Operations: ",operList)

    total_comp=0
    ans_list=[]
    dict = unigramInvIndex()
    for i in range(0,len(ans)-1):
        op=operList[i]
        word1=ans[i]
        word2=ans[i+1]
        comp=0
        opList=[]
        if dict.get(word1) is not None:
            list1=dict[word1]
        else:
            list1 = []
        if dict.get(word2) is not None:
            list2=dict[word2]
        else:
            list2 = []

        ##### Query Oprations #######
        if op == "OR":
            if(len(ans_list)!=0):
                list1=ans_list
            opList,cmpr=orFunc(list1, list2)
            ans_list=opList
            comp+=cmpr
        elif op == "AND":
            if(len(ans_list)!=0):
                list1=ans_list
            opList,cmpr=andFunc(list1, list2)
            ans_list=opList
            comp+=cmpr
        elif op == "OR NOT":
            if(len(ans_list)!=0):
                list1=ans_list
            opList,cmpr=orNot(list1, list2)
            ans_list=opList
            comp+=cmpr
        elif op == "AND NOT":
            if(len(ans_list)!=0):
                list1=ans_list
            opList,cmpr=andNot(list1, list2)
            ans_list=opList
            comp+=cmpr
        total_comp+=comp

    ## Query ##
    ip_format=""
    for i in range(0,len(ans)-1):
        ip_format+=ans[i]+" "+operList[i]+" "
    ip_format+=ans[len(ans)-1]

    ## Doc Names ##
    ans_doc_name=[]
    for i in ans_list:
        ans_doc_name.append(doctIdDict[i])

    ######## Output #########
    
    print("Query : ",ip_format)
    print("No of documents retrieved: ", len(ans_list))
    print("Names of document retrieved: ",ans_doc_name)
    print("Minimum Number comaparisons required: ", total_comp)

In [21]:
n=int(input("Enter number of queries: "))
for i in range (n):
    userInput()

Enter Query: 
Enter 2 Operations: 
Tokens : ['experimental', 'solutions', 'slipstram']
Operations:  ['AND', 'OR']
Query :  experimental AND solutions OR slipstram
No of documents retrieved:  20
Names of document retrieved:  ['cranfield0047', 'cranfield0188', 'cranfield0329', 'cranfield0435', 'cranfield0467', 'cranfield0498', 'cranfield0518', 'cranfield0576', 'cranfield0644', 'cranfield0663', 'cranfield0717', 'cranfield0753', 'cranfield0767', 'cranfield0823', 'cranfield1074', 'cranfield1078', 'cranfield1083', 'cranfield1185', 'cranfield1214', 'cranfield1352']
Minimum Number comaparisons required:  421
